Necessary Library Imports

In [ ]:
import random
import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers

Mount the Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Read the data file

In [ ]:
import csv
import random

csv_file = "/content/drive/MyDrive/MiniProject03/data.csv"

# Read the CSV file into a list of rows
with open(csv_file, 'r') as file:
    csv_reader = list(csv.reader(file))

# Shuffle the rows randomly
random.shuffle(csv_reader)

# Process the shuffled rows, limiting to 20 lines
for i, row in enumerate(csv_reader[:20], start=1):
    cell1, cell2 = row
    print(f"Row {i}: {cell1}\t{cell2}")


Row 1: I play the bass.	මම බාස් වාදනය කරනවා.
Row 2: Are you embarrassed?	ඔබ ලැජ්ජාවට පත් වෙනවාද?
Row 3: I'm invited to a party tonight.	මම අද රෑ සාදයකට ආරාධනා කරෙයි.
Row 4: It isn't easy to write a love letter in English.	ආදර ලිපියක් ඉංග්රීසියෙන් ලිවීම පහසු නැත.
Row 5: The dog kept barking all through the night.	බල්ලා රාත්රිය පුරාම බාධාවක් විය.
Row 6: Tom disappointed Mary.	ටොම් මරියා කලකිරීමට පත් විය.
Row 7: Don't smoke in this room.	මේ කාමරයේ දුම් බොන්න එපා.
Row 8: I gave my father a silk tie.	මම මගේ පියාට සේද ටයි පටියක් දුන්නා.
Row 9: Tom is different from other boys.	ටොම් අනෙක් පිරිමි ළමයින්ට වඩා වෙනස් ය.
Row 10: That plane is enormous!	එම යානය අතිමහත් ය!
Row 11: I want you to get in the car.	මට ඕන ඔයා කාර් එකට නගින්න.
Row 12: He was expelled from school.	ඔහු පාසලෙන් නෙරපා හරින ලදී.
Row 13: He doesn't like to lose.	ඔහු පරාජය වීමට කැමති නැත.
Row 14: I didn't go to school yesterday.	මම ඊයේ පාසලට ගියේ නැහැ.
Row 15: Doesn't that make you happy?	එය ඔබව සතුටු කරන්නේ නැද්ද?
Row 16: Why do

Split the English and Sinhala translation pairs

In [ ]:
csv_file = "/content/drive/MyDrive/MiniProject03/data.csv"

text_pairs = []
with open(csv_file, 'r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        if len(row) >= 2 and row[1] is not None:
            english, sinhala = row[:2]
            sinhala = "[start] " + sinhala + " [end]"
            text_pairs.append((english, sinhala))

for _ in range(5):
    print(random.choice(text_pairs))


('You might want to talk to Tom.', '[start] ඔබට ටොම් සමඟ කතා කිරීමට අවශ්ය විය හැකිය. [end]')
('She makes her mother happy.', '[start] ඇය මව සතුටට පත් කරයි. [end]')
('My father is sweeping the garage.', '[start] මගේ තාත්තා ගරාජය අතුගා දමයි. [end]')
('There are many parks in London.', '[start] ලන්ඩනයේ බොහෝ උද්යාන තිබේ. [end]')
('They took Tom away on a stretcher.', '[start] ඔවුන් ස්ට්රෙචරයක් මත ටොම්ව රැගෙන ගියා. [end]')


Randomize the data

In [ ]:
random.shuffle(text_pairs)

Spliting the data into training, validation and Testing

In [ ]:
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

print("Total sentences:",len(text_pairs))
print("Training set size:",len(train_pairs))
print("Validation set size:",len(val_pairs))
print("Testing set size:",len(test_pairs))


Total sentences: 102903
Training set size: 72033
Validation set size: 15435
Testing set size: 15435


In [ ]:
len(train_pairs)+len(val_pairs)+len(test_pairs)

102903

Removing Punctuations

In [ ]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

f"[{re.escape(strip_chars)}]"

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\\\\\^_`\\{\\|\\}\\~¿]'

Vectorizing the English and Sinhala text pairs

In [ ]:
def custom_standardization(input_string):
  lowercase = tf.strings.lower(input_string)
  return tf.strings.regex_replace(
      lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_sinhala_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_sinhala_texts)


Preparing datasets for the translation task

In [ ]:
batch_size = 64

def format_dataset(eng, sin):
   eng = source_vectorization(eng)
   sin = target_vectorization(sin)
   return ({
         "english": eng,
         "sinhala": sin[:, :-1],
   }, sin[:, 1:])

def make_dataset(pairs):
    eng_texts, sin_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    sin_texts = list(sin_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, sin_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)


for inputs, targets in train_ds.take(1):
   print(f"inputs['english'].shape: {inputs['english'].shape}")
   print(f"inputs['sinhala'].shape: {inputs['sinhala'].shape}")
   print(f"targets.shape: {targets.shape}")




inputs['english'].shape: (64, 20)
inputs['sinhala'].shape: (64, 20)
targets.shape: (64, 20)


In [ ]:
print(list(train_ds.as_numpy_iterator())[50])

({'english': array([[   9,   70,  305, ...,    0,    0,    0],
       [ 313,  101,    0, ...,    0,    0,    0],
       [   5,   32,   24, ...,    0,    0,    0],
       ...,
       [  27, 3057,  583, ...,    0,    0,    0],
       [   3,   99,  987, ...,    0,    0,    0],
       [4349, 2082,   49, ...,    0,    0,    0]]), 'sinhala': array([[   2,    8,  861, ...,    0,    0,    0],
       [   2,  135,  865, ...,    0,    0,    0],
       [   2,    5,   58, ...,    0,    0,    0],
       ...,
       [   2,   16, 8494, ...,    0,    0,    0],
       [   2,    4,   10, ...,    0,    0,    0],
       [   2,  889,  218, ...,    0,    0,    0]])}, array([[   8,  861,  441, ...,    0,    0,    0],
       [ 135,  865,    3, ...,    0,    0,    0],
       [   5,   58,  101, ...,    0,    0,    0],
       ...,
       [  16, 8494,  648, ...,    0,    0,    0],
       [   4,   10, 4955, ...,    0,    0,    0],
       [ 889,  218, 1957, ...,    0,    0,    0]]))


Transformer encoder implemented as a subclassed Layer

In [ ]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
      super().__init__(**kwargs)
      self.embed_dim = embed_dim
      self.dense_dim = dense_dim
      self.num_heads = num_heads
      self.attention = layers.MultiHeadAttention(
           num_heads=num_heads, key_dim=embed_dim)
      self.dense_proj = keras.Sequential(
           [layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),]
      )
      self.layernorm_1 = layers.LayerNormalization()
      self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
      if mask is not None:
         mask = mask[:, tf.newaxis, :]
      attention_output = self.attention(
         inputs, inputs, attention_mask=mask)
      proj_input = self.layernorm_1(inputs + attention_output)
      proj_output = self.dense_proj(proj_input)
      return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config


The Transformer decoder

In [ ]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
      super().__init__(**kwargs)
      self.embed_dim = embed_dim
      self.dense_dim = dense_dim
      self.num_heads = num_heads
      self.attention_1 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
      self.attention_2 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
      self.dense_proj = keras.Sequential(
          [layers.Dense(dense_dim, activation="relu"),
           layers.Dense(embed_dim),]
      )
      self.layernorm_1 = layers.LayerNormalization()
      self.layernorm_2 = layers.LayerNormalization()
      self.layernorm_3 = layers.LayerNormalization()
      self.supports_masking = True

    def get_config(self):
      config = super().get_config()
      config.update({
          "embed_dim": self.embed_dim,
          "num_heads": self.num_heads,
          "dense_dim": self.dense_dim,
      })
      return config

    def get_causal_attention_mask(self, inputs):
      input_shape = tf.shape(inputs)
      batch_size, sequence_length = input_shape[0], input_shape[1]
      i = tf.range(sequence_length)[:, tf.newaxis]
      j = tf.range(sequence_length)
      mask = tf.cast(i >= j, dtype="int32")
      mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
      mult = tf.concat(
              [tf.expand_dims(batch_size, -1),
               tf.constant([1, 1], dtype=tf.int32)], axis=0)
      return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
      causal_mask = self.get_causal_attention_mask(inputs)
      if mask is not None:
           padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
           padding_mask = tf.minimum(padding_mask, causal_mask)
      else:
           padding_mask = mask
      attention_output_1 = self.attention_1(
           query=inputs,
           value=inputs,
           key=inputs,
           attention_mask=causal_mask)
      attention_output_1 = self.layernorm_1(inputs + attention_output_1)
      attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
       )
      attention_output_2 = self.layernorm_2(
          attention_output_1 + attention_output_2)
      proj_output = self.dense_proj(attention_output_2)
      return self.layernorm_3(attention_output_2 + proj_output)



Positional Encoding

In [ ]:
class PositionalEmbedding(layers.Layer):
     def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
         super().__init__(**kwargs)
         self.token_embeddings = layers.Embedding(
             input_dim=input_dim, output_dim=output_dim)
         self.position_embeddings = layers.Embedding(
             input_dim=sequence_length, output_dim=output_dim)
         self.sequence_length = sequence_length
         self.input_dim = input_dim
         self.output_dim = output_dim

     def call(self, inputs):
         length = tf.shape(inputs)[-1]
         positions = tf.range(start=0, limit=length, delta=1)
         embedded_tokens = self.token_embeddings(inputs)
         embedded_positions = self.position_embeddings(positions)
         return embedded_tokens + embedded_positions

     def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

     def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
             "output_dim": self.output_dim,
             "sequence_length": self.sequence_length,
             "input_dim": self.input_dim,
        })
        return config


End-to-end Transformer

In [ ]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="sinhala")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [ ]:
from tensorflow import keras
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import TerminalTrueColorFormatter

# Get the summary text
summary_text = []
transformer.summary(print_fn=lambda x: summary_text.append(x))

# Highlight the summary text using Pygments
highlighted_summary = highlight('\n'.join(summary_text), PythonLexer(), TerminalTrueColorFormatter(style='monokai'))

# Print the highlighted summary
print(highlighted_summary)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 english (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 sinhala (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 positional_embedding (Posi  (None, None, 256)            3845120   ['english[0][0]']             
 tionalEmbedding)                                                                                 
                                                                                                  
 positional_embedding_1 (Po  (None, None, 256)            3845120   ['sinhala[0][0]']         

Training the sequence-to-sequence Transformer

In [ ]:
from tensorflow.keras.optimizers import Adam

# Compile the model with Adam optimizer
transformer.compile(
    optimizer=Adam(),  # Using Adam optimizer
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

# Train the model
history = transformer.fit(train_ds, epochs=30, validation_data=val_ds)

# Save the model
transformer.save("transformer_model")


Epoch 1/30
1126/1126 [==============================] - 113s 90ms/step - loss: 2.2349 - accuracy: 0.6233 - val_loss: 2.1400 - val_accuracy: 0.6307
Epoch 2/30
1126/1126 [==============================] - 81s 72ms/step - loss: 1.8248 - accuracy: 0.6619 - val_loss: 1.9965 - val_accuracy: 0.6442
Epoch 3/30
1126/1126 [==============================] - 82s 73ms/step - loss: 1.5342 - accuracy: 0.6917 - val_loss: 1.9253 - val_accuracy: 0.6540
Epoch 4/30
1126/1126 [==============================] - 84s 75ms/step - loss: 1.3261 - accuracy: 0.7166 - val_loss: 1.9221 - val_accuracy: 0.6596
Epoch 5/30
1126/1126 [==============================] - 82s 73ms/step - loss: 1.1637 - accuracy: 0.7391 - val_loss: 1.9260 - val_accuracy: 0.6602
Epoch 6/30
1126/1126 [==============================] - 84s 75ms/step - loss: 1.0386 - accuracy: 0.7579 - val_loss: 1.9236 - val_accuracy: 0.6662
Epoch 7/30
1126/1126 [==============================] - 82s 73ms/step - loss: 0.9384 - accuracy: 0.7761 - val_loss: 1.9805 

Translating new sentences with our Transformer model

In [20]:
sin_vocab = target_vectorization.get_vocabulary()
sin_index_lookup = dict(zip(range(len(sin_vocab)), sin_vocab))
max_decoded_sentence_length = 30


Output Testing and Decoding the output sequence

In [21]:
import numpy as np

def decode_sequence(input_sentence):
     tokenized_input_sentence = source_vectorization([input_sentence])
     decoded_sentence = "[start]"
     for i in range(max_decoded_sentence_length):
       tokenized_target_sentence = target_vectorization(
         [decoded_sentence])[:, :-1]
       predictions = transformer(
         [tokenized_input_sentence, tokenized_target_sentence])
       sampled_token_index = np.argmax(predictions[0, i, :])
       sampled_token = sin_index_lookup[sampled_token_index]
       decoded_sentence += " " + sampled_token
       if sampled_token == "[end]": break
     return decoded_sentence

Transformer translating output

In [22]:
test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(10):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
It is not a watch.
[start] එය ඔරලෝසුවක් නොවේ [end]
-
Ask him his name.
[start] ඔහුගේ නම අහන්න [end]
-
Is it private?
[start] එය පෞද්ගලික [UNK] [end]
-
I smiled.
[start] මම සිනාසුණේය [end]
-
Tom handed a note to Mary.
[start] ටොම් මලක් ගෙනැවිත් ටෙම්ට [end]
-
The grapefruit tastes very sour.
[start] ගෙඩි ඉතා ඇඹුල් රසයි [end]
-
Tom is going to break up with Mary.
[start] ටොම් මරියා සමඟ පිටතට යාමට යන්නේ ය [end]
-
I can't remember.
[start] මට මතකයි [end]
-
It's a classic.
[start] එය [UNK] [end]
-
Tom has only done half his homework.
[start] ටොම් සිය ගෙදර වැඩ කටයුතු සඳහා කර ඇත [end]


Evaluation using the BLEU score

In [23]:
from nltk.translate.bleu_score import sentence_bleu

test_eng_texts = [pair[0] for pair in test_pairs]
test_sin_texts = [pair[1] for pair in test_pairs]
score = 0
bleu  = 0
for i in range(20):
    candidate = decode_sequence(test_eng_texts[i])
    reference = test_sin_texts[i].lower()
    print(candidate,reference)
    score = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
    bleu += score
    print(f"Score:{score}")
scaled_bleu = (bleu / 20) * 100
print(f"\nScaled BLEU score : {round(scaled_bleu,2)}/100")


[start] ටොම් බාල්දිය වතුරෙන් පුරවා ගත්තේය [end] [start] ටොම් බාල්දිය ඉහළට පුරවා ඇත. [end]
Score:0.5319148936170213


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

[start] එලාම් සක්රිය කිරීම සඳහා මෙම මෘදු the ය යුතුය [end] [start] සංකේතාංකන තාක්ෂණය එය තරමක් විශ්වාසදායක තැන දක්වා ඉදිරියට ගොස් ඇත. [end]
Score:0.3620689655172414
[start] මෙය දුෂ්කර බව මම දනිමි [end] [start] මම දන්නවා මේක අමාරුයි කියලා. [end]
Score:0.5555555555555556
[start] අපි මෙම ආකාරයේ බළලෙකුට අක්ෂර වින්යාසයක් දමන්න [end] [start] මෙම ආකාරයේ සිදුවීමක් පුනරාවර්තනය වීම අප වළක්වා ගත යුතුය. [end]
Score:0.4745762711864407
[start] මම මගේ නිවාඩුව දෙස බලා ගත්තා [end] [start] මම මගේ නිවාඩුව හකාන් එකක ගත කළා. [end]
Score:0.5
[start] මම හිතුවේ ඔයාට ටොම්ට කරන්න ඕන කියලා [end] [start] මම හිතුවේ ඔයාට ටොම් සමඟ සාදයට යන්න ඕන කියලා. [end]
Score:0.5510204081632653
[start] අපි කෑම වලින් එළියට ගියා [end] [start] අපි කෑමෙන් ඉවරයි. [end]
Score:0.5263157894736842
[start] එය කාලය කුමක්ද කියා මම කල්පනා කරමි [end] [start] මම කල්පනා කරන්නේ එය කුමන වේලාවක්ද යන්නයි. [end]
Score:0.4791666666666667
[start] මතක තබා ගැනීම නවත්වන්න [end] [start] කලබල වීම නවත්වන්න. [end]
Score:0.5
[start] මේක ගොඩක් නරකයි [end] [star

Calculate accuracy for 20 new sentences

In [24]:
manualTest = [
    ("I'd like to send my dress to a dry cleaner.", "මම මගේ ඇඳුම වියළි පිරිසිදු කරන්නෙකුට යැවීමට කැමතියි."),
    ("Tom is ridiculous.", "ටොම් හාස්යජනකයි."),
    ("That's quite remarkable.", "එය තරමක් කැපී පෙනේ."),
    ("Tom wasn't happy.", "ටොම් සතුටු නොවීය."),
    ("I've never woken up this early.", "මම මේ වේලාසනින් මේ කවදාවත් අවදි කළේ නැහැ."),
    ("Harshness should be avoided in those cases.", "එම අවස්ථාවන්හිදී රළුබව වළක්වා ගත යුතුය."),
    ("He hid his sadness behind a smile.", "ඔහු තම දුක සිනහවක් පිටුපස සඟවා ගත්තේය."),
    ("Tom currently doesn't make as much money as he used to.", "ටොම් දැනට ඔහු පුරුදු වී ඇති තරම් මුදල් උපයන්නේ නැත."),
    ("I owe my success to her help.", "ඇගේ උදව්වට මගේ සාර්ථකත්වයට මම ණයගැතියි."),
    ("Some stores discount the price.", "සමහර වෙළඳසැල් මඟින් මිල වට්ටම් කරයි."),
    ("She installed a new electric stove in the room.", "ඇය කාමරයේ නව විදුලි උදුනක් සවි කළාය."),
    ("I have failed.", "මම අසමත් වුණා."),
    ("Let's ask if it's true.", "එය සත්ය දැයි විමසමු."),
    ("I can't understand what you're saying.", "මට තේරෙනවා ඔයා කියන දේ."),
    ("I want to be a magician.", "මට ඉන්ද්රජාලිකයෙකු වීමට අවශ්යයි."),
    ("We shall leave tomorrow, weather permitting.", "අපි හෙට පිටත් වෙමු, කාලගුණික අවසරය."),
    ("Do you know why spring rolls are called spring rolls?", "වසන්ත රෝල් වසන්ත රෝල් ලෙස හැඳින්වේ ඇයි කියා ඔබ දන්නවාද?"),
    ("Have you been totally honest with me?", "ඔබ මා සමඟ මුළුමනින්ම අවංක වී සිටියාද?"),
    ("Money rules the world.", "මුදල් ලෝකය පාලනය කරයි."),
    ("Practice is the only way to master foreign languages.", "විදේශ භාෂා ප්රගුණ කිරීමට ඇති එකම ක්රමය පුරුද්දයි.")
]



from nltk.translate.bleu_score import sentence_bleu

testENTexts = [pair[0] for pair in manualTest]
testSITexts = [pair[1] for pair in manualTest]
scores = []
bleu  = 0
for i in range(20):
    candidate = decode_sequence(testENTexts[i])
    reference = testSITexts[i].lower()
    print(candidate, reference)
    score = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
    scores.append(score)
    bleu += score
    print(f"Score: {score}")
scaled_bleu = (bleu / 20) * 100
print(f"\nScaled BLEU score : {round(scaled_bleu, 2)}/100")


[start] මම මගේ ඇඳුම වියළි පිරිසිදු [UNK] යැවීමට කැමතියි [end] මම මගේ ඇඳුම වියළි පිරිසිදු කරන්නෙකුට යැවීමට කැමතියි.
Score: 0.32786885245901637
[start] ටොම් හාස්යජනක වේ [end] ටොම් හාස්යජනකයි.
Score: 0.4
[start] එය තරමක් කැපී පෙනේ [end] එය තරමක් කැපී පෙනේ.
Score: 0.4375
[start] ටොම් සතුටු නොවීය [end] ටොම් සතුටු නොවීය.
Score: 0.4
[start] මම මීට ඉක්මනින් කවදාවත් මේ වගේ නැහැ [end] මම මේ වේලාසනින් මේ කවදාවත් අවදි කළේ නැහැ.
Score: 0.2653061224489796
[start] එම අවස්ථාවන්හිදී රළුබව වළක්වා ගත යුතුය [end] එම අවස්ථාවන්හිදී රළුබව වළක්වා ගත යුතුය.
Score: 0.4230769230769231
[start] ඔහු තම දුක සිනහවක් පිටුපස සඟවා ගත්තේය [end] ඔහු තම දුක සිනහවක් පිටුපස සඟවා ගත්තේය.
Score: 0.39215686274509803
[start] ටොම් දැනට ප්රමාණවත් තරම් මුදල් උපයන්නේ නැත [end] ටොම් දැනට ඔහු පුරුදු වී ඇති තරම් මුදල් උපයන්නේ නැත.
Score: 0.30357142857142855
[start] මගේ උපකාරය මට මගේ සාර්ථකත්වයට ණයගැතියි [end] ඇගේ උදව්වට මගේ සාර්ථකත්වයට මම ණයගැතියි.
Score: 0.3461538461538461
[start] සමහර වෙළඳසැල් මඟින් මිල වට්ටම් කරයි [end] සමහර වෙළඳසැල